In [11]:
import com.toldoven.aoc.notebook.AocClient

val aoc = AocClient.fromEnv().interactiveDay(2020, 19)

aoc.viewPartOne()

--- Day 19: Monster Messages --- 
 You land in an airport surrounded by dense forest. As you walk to your high-speed train, the Elves at the Mythical Information Bureau contact you again. They think their satellite has collected an image of a sea monster ! Unfortunately, the connection to the satellite is having problems, and many of the messages sent back from the satellite have been corrupted. 
 They sent you a list of the rules valid messages should obey and a list of received messages they've collected so far (your puzzle input). 
 The rules for valid messages (the top part of your puzzle input) are numbered and build upon each other. For example: 
 0: 1 2
1: "a"
2: 1 3 | 3 1
3: "b"
 
 Some rules, like 3: "b" , simply match a single character (in this case, b ). 
 The remaining rules list the sub-rules that must be followed; for example, the rule 0: 1 2 means that to match rule 0 , the text being checked must match rule 1 , and the text after the part that matched rule 1 must then match rule 2 . 
 Some of the rules have multiple lists of sub-rules separated by a pipe ( | ). This means that at least one list of sub-rules must match. (The ones that match might be different each time the rule is encountered.) For example, the rule 2: 1 3 | 3 1 means that to match rule 2 , the text being checked must match rule 1 followed by rule 3 or it must match rule 3 followed by rule 1 . 
 Fortunately, there are no loops in the rules, so the list of possible matches will be finite. Since rule 1 matches a and rule 3 matches b , rule 2 matches either ab or ba . Therefore, rule 0 matches aab or aba . 
 Here's a more interesting example: 
 0: 4 1 5
1: 2 3 | 3 2
2: 4 4 | 5 5
3: 4 5 | 5 4
4: "a"
5: "b"
 
 Here, because rule 4 matches a and rule 5 matches b , rule 2 matches two letters that are the same ( aa or bb ), and rule 3 matches two letters that are different ( ab or ba ). 
 Since rule 1 matches rules 2 and 3 once each in either order, it must match two pairs of letters, one pair with matching letters and one pair with different letters. This leaves eight possibilities: aaab , aaba , bbab , bbba , abaa , abbb , baaa , or babb . 
 Rule 0 , therefore, matches a (rule 4 ), then any of the eight options from rule 1 , then b (rule 5 ): aaaabb , aaabab , abbabb , abbbab , aabaab , aabbbb , abaaab , or ababbb . 
 The received messages (the bottom part of your puzzle input) need to be checked against the rules so you can determine which are valid and which are corrupted. Including the rules and the messages together, this might look like: 
 0: 4 1 5
1: 2 3 | 3 2
2: 4 4 | 5 5
3: 4 5 | 5 4
4: "a"
5: "b"

ababbb
bababa
abbbab
aaabbb
aaaabbb
 
 Your goal is to determine the number of messages that completely match rule 0 . In the above example, ababbb and abbbab match, but bababa , aaabbb , and aaaabbb do not, producing the answer 2 . The whole message must match all of rule 0 ; there can't be extra unmatched characters in the message. (For example, aaaabbb might appear to match rule 0 above, but it has an extra unmatched b on the end.) 
 How many messages completely match rule 0 ?

In [12]:
val input = aoc.input()
input

[124: 72 26 | 58 91, 76: 72 90 | 58 73, 89: 58 135 | 72 25, 43: 58 30 | 72 98, 130: 58 58, 87: 135 72 | 100 58, 24: 72 5 | 58 78, 61: 84 72 | 71 58, 8: 42, 68: 49 72 | 91 58, 103: 126 58 | 124 72, 132: 58 17 | 72 37, 75: 72 89 | 58 50, 99: 72 72 | 58 58, 28: 58 19 | 72 65, 2: 9 72 | 67 58, 113: 58 100 | 72 130, 66: 129 58 | 70 72, 74: 72 116 | 58 56, 45: 72 82 | 58 38, 70: 137 58 | 12 72, 78: 72 58 | 58 58, 115: 78 58 | 27 72, 48: 58 121 | 72 95, 23: 120 58 | 93 72, 44: 58 25 | 72 130, 17: 58 27 | 72 91, 65: 102 72 | 46 58, 19: 72 84 | 58 35, 79: 29 72 | 127 58, 13: 58 88 | 72 136, 6: 15 72 | 130 58, 81: 58 72, 100: 117 72 | 58 58, 116: 78 72 | 49 58, 136: 20 72 | 50 58, 97: 53 72 | 85 58, 107: 58 27 | 72 5, 0: 8 11, 49: 58 72 | 72 117, 88: 105 58 | 21 72, 83: 69 72 | 5 58, 112: 58 32 | 72 2, 47: 62 72 | 118 58, 59: 58 134 | 72 101, 60: 132 72 | 103 58, 55: 58 44 | 72 92, 67: 72 15 | 58 100, 96: 81 72 | 135 58, 121: 25 58 | 26 72, 39: 58 81 | 72 25, 82: 81 58, 27: 72 72 | 117 58, 85: 5

In [13]:
val example = """0: 4 1 5
1: 2 3 | 3 2
2: 4 4 | 5 5
3: 4 5 | 5 4
4: "a"
5: "b"

ababbb
bababa
abbbab
aaabbb
aaaabbb""".split("\n")

In [16]:
sealed interface Rule {
    fun matches(s: String): Int;

}

class StringRule(val rule: String) : Rule {
    override fun matches(s: String): Int {
        return if (s.startsWith(rule)) rule.length else 0
    }
}

class CompositeRule(val rules: List<Rule>) : Rule {
    override fun matches(s: String): Int {
        var curIdx = 0
        rules.forEach { rule ->
            val result = rule.matches(s.substring(curIdx))
            if (result == 0) {
                return 0
            }
            curIdx += result
        }
        return curIdx
    }
}

class RuleReference(val rulesMap: Map<Int, Rule>, val ruleNum: Int) : Rule {

    override fun matches(s: String): Int {
        return rulesMap[ruleNum]!!.matches(s)
    }
}

class OrRule(val rules: List<Rule>) : Rule {

    override fun matches(s: String): Int {
        return rules.map { rule -> rule.matches(s) }
            .filter { it != 0 }
            .first().or(0)
    }
}
    

In [20]:
fun parseCompositeRule(rulesStr: String, map: MutableMap<Int, Rule>): CompositeRule {
    val rules = rulesStr.trim().split(" ")
    val toUse = rules.map { ruleStr ->
        RuleReference(map, ruleStr.toInt())
    }
    return CompositeRule(toUse)
}

fun parse(lines: List<String>): Pair<Map<Int, Rule>, List<String>> {
    var curLine = 0
    val rules = mutableMapOf<Int, Rule>()
    while (lines[curLine].isNotBlank()) {
        val line = lines[curLine]
        val num = line.split(":")[0].toInt()
        val rulesStr = line.split(":")[1].trim()
        if (rulesStr.contains("|")) {
            val rulesInOr = rulesStr.split("|")
                .map { it.trim() }
                .map { parseCompositeRule(it, rules) }
                .toList()
            rules[num] = OrRule(rulesInOr)
        } else if ("\"" in rulesStr) {
        
            rules[num] = StringRule(rulesStr.trim('"')).also { println(it.rule) }
        } else {
            rules[num] = parseCompositeRule(rulesStr, rules)
        }
        curLine++
    }

    val messages = lines.subList(curLine + 1, lines.size)
    return Pair(rules, messages)
}

val (rules, messages) = parse(example)

rules.size

a
b


6

In [21]:
aoc.viewPartTwo()

--- Part Two --- 
 As you look over the list of messages, you realize your matching rules aren't quite right. To fix them, completely replace rules 8: 42 and 11: 42 31 with the following: 
 8: 42 | 42 8
11: 42 31 | 42 11 31
 
 This small change has a big impact: now, the rules do contain loops, and the list of messages they could hypothetically match is infinite. You'll need to determine how these changes affect which messages are valid. 
 Fortunately, many of the rules are unaffected by this change; it might help to start by looking at which rules always match the same set of values and how those rules (especially rules 42 and 31 ) are used by the new versions of rules 8 and 11 . 
 (Remember, you only need to handle the rules you have ; building a solution that could handle any hypothetical combination of rules would be significantly more difficult .) 
 For example: 
 42: 9 14 | 10 1
9: 14 27 | 1 26
10: 23 14 | 28 1
1: "a"
11: 42 31
5: 1 14 | 15 1
19: 14 1 | 14 14
12: 24 14 | 19 1
16: 15 1 | 14 14
31: 14 17 | 1 13
6: 14 14 | 1 14
2: 1 24 | 14 4
0: 8 11
13: 14 3 | 1 12
15: 1 | 14
17: 14 2 | 1 7
23: 25 1 | 22 14
28: 16 1
4: 1 1
20: 14 14 | 1 15
3: 5 14 | 16 1
27: 1 6 | 14 18
14: "b"
21: 14 1 | 1 14
25: 1 1 | 1 14
22: 14 14
8: 42
26: 14 22 | 1 20
18: 15 15
7: 14 5 | 1 21
24: 14 1

abbbbbabbbaaaababbaabbbbabababbbabbbbbbabaaaa
bbabbbbaabaabba
babbbbaabbbbbabbbbbbaabaaabaaa
aaabbbbbbaaaabaababaabababbabaaabbababababaaa
bbbbbbbaaaabbbbaaabbabaaa
bbbababbbbaaaaaaaabbababaaababaabab
ababaaaaaabaaab
ababaaaaabbbaba
baabbaaaabbaaaababbaababb
abbbbabbbbaaaababbbbbbaaaababb
aaaaabbaabaaaaababaa
aaaabbaaaabbaaa
aaaabbaabbaaaaaaabbbabbbaaabbaabaaa
babaaabbbaaabaababbaabababaaab
aabbbbbaabbbaaaaaabbbbbababaaaaabbaaabba
 
 Without updating rules 8 and 11 , these rules only match three messages: bbabbbbaabaabba , ababaaaaaabaaab , and ababaaaaabbbaba . 
 However, after updating rules 8 and 11 , a total of 12 messages match: 
 
 bbabbbbaabaabba 
 babbbbaabbbbbabbbbbbaabaaabaaa 
 aaabbbbbbaaaabaababaabababbabaaabbababababaaa 
 bbbbbbbaaaabbbbaaabbabaaa 
 bbbababbbbaaaaaaaabbababaaababaabab 
 ababaaaaaabaaab 
 ababaaaaabbbaba 
 baabbaaaabbaaaababbaababb 
 abbbbabbbbaaaababbbbbbaaaababb 
 aaaaabbaabaaaaababaa 
 aaaabbaabbaaaaaaabbbabbbaaabbaabaaa 
 aabbbbbaabbbaaaaaabbbbbababaaaaabbaaabba 
 
 After updating rules 8 and 11 , how many messages completely match rule 0 ?